# ProyectoParteIII_TorresRomero

Este notebook forma parte del proyecto final y aplica técnicas de selección de variables, clasificación y evaluación de modelo sobre el dataset de defunciones por accidentes de tránsito.

## 1. Carga de datos y preparación

En esta sección se carga el archivo original con los datos de defunciones por accidentes de tránsito. A partir del DataFrame completo, se seleccionan las columnas relevantes para el modelo predictivo. Se eliminan los registros con valores nulos para asegurar un entrenamiento limpio y consistente.

In [18]:
import pandas as pd

# Cargar archivo base
df_defunciones = pd.read_excel("/content/BDD defunciones-incidentes-viales-2017.xlsx")

# Variables seleccionadas para el modelo
columnas_utiles = ['Edad', 'Momento del hecho', 'Vehiculo victima', 'Via', 'Clima']
df_modelo = df_defunciones[columnas_utiles + ['Sexo']].dropna()
df_modelo.head()

,Edad,Momento del hecho,Vehiculo victima,Via,Clima,Sexo
0,75,Sin especificar,Sin especificar,Sin especificar,Sin especificar,Masculino
1,21,Diurno,Vehiculo,Ruta provincial,Bueno,Masculino
2,22,Nocturno,Motocicleta,Calle,Sin especificar,Masculino
3,42,Diurno,Vehiculo,Ruta provincial,Bueno,Masculino
4,18,Nocturno,Motocicleta,Calle,Bueno,Masculino


## 2. Codificación de variables categóricas

Como los algoritmos de machine learning no trabajan directamente con texto, es necesario transformar las variables categóricas (como 'Momento del hecho' o 'Vehiculo victima') en valores numéricos. Se utiliza la técnica de `Label Encoding`, que asigna un número entero a cada categoría. Esta codificación es adecuada cuando no hay un orden natural entre las categorías, como en este caso.


In [19]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

df_encoded = df_modelo.copy()

# Handle 'Sin especificar' in 'Edad' column
df_encoded['Edad'] = df_encoded['Edad'].replace('Sin especificar', np.nan)
df_encoded['Edad'] = pd.to_numeric(df_encoded['Edad'], errors='coerce')
df_encoded.dropna(subset=['Edad'], inplace=True)


for col in ['Momento del hecho', 'Vehiculo victima', 'Via', 'Clima', 'Sexo']:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df_encoded[col])

df_encoded.head()

/tmp/ipython-input-19-2332227405.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_encoded['Edad'] = df_encoded['Edad'].replace('Sin especificar', np.nan)


,Edad,Momento del hecho,Vehiculo victima,Via,Clima,Sexo
0,75.0,3,8,7,5,1
1,21.0,0,9,6,0,1
2,22.0,1,5,2,5,1
3,42.0,0,9,6,0,1
4,18.0,1,5,2,0,1


## 3. División del dataset

Aquí se divide el dataset en dos conjuntos: uno para entrenamiento (70%) y otro para prueba (30%). Esta separación permite entrenar el modelo con una parte de los datos y luego evaluarlo con datos que nunca ha visto, lo que ayuda a estimar su capacidad de generalización. La función `train_test_split` de `sklearn` se usa para este propósito, asegurando aleatoriedad y reproducibilidad con `random_state`.


In [20]:
from sklearn.model_selection import train_test_split
import numpy as np

# Handle 'Sin especificar' in 'Edad' column and drop rows with NaN
df_cleaned = df_modelo.copy()
df_cleaned['Edad'] = df_cleaned['Edad'].replace('Sin especificar', np.nan)
df_cleaned['Edad'] = pd.to_numeric(df_cleaned['Edad'], errors='coerce')
df_cleaned.dropna(subset=['Edad'], inplace=True)

# Apply Label Encoding to categorical columns *after* cleaning 'Edad'
for col in ['Momento del hecho', 'Vehiculo victima', 'Via', 'Clima', 'Sexo']:
    le = LabelEncoder()
    df_cleaned[col] = le.fit_transform(df_cleaned[col])


X = df_cleaned.drop('Sexo', axis=1)
y = df_cleaned['Sexo']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

/tmp/ipython-input-20-2841728554.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_cleaned['Edad'] = df_cleaned['Edad'].replace('Sin especificar', np.nan)


## 4. Entrenamiento del modelo de clasificación (Árbol de Decisión)

Se entrena un modelo de árbol de decisión, un algoritmo de clasificación que segmenta el espacio de datos en función de las variables seleccionadas, generando reglas que permiten predecir la clase (en este caso, el sexo de la víctima). El parámetro `max_depth` controla la profundidad del árbol para evitar sobreajuste. Este modelo es fácil de interpretar y rápido de entrenar, ideal para esta primera aproximación.


In [23]:
from sklearn.tree import DecisionTreeClassifier

modelo = DecisionTreeClassifier(max_depth=5, random_state=42)
modelo.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=5, random_state=42)

## 5. Evaluación del modelo

En esta sección se evalúa el rendimiento del modelo utilizando métricas como `accuracy`, `precision`, `recall` y la `matriz de confusión`. Estas métricas permiten medir qué tan bien el modelo predice el sexo de las víctimas, detectando posibles desbalances o errores comunes. Una buena evaluación es esencial para validar que el modelo no solo aprende, sino que también generaliza.


In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

y_pred = modelo.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("Matriz de Confusión:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.7286245353159851
Precision: 0.7782258064516129
Recall: 0.9146919431279621
Matriz de Confusión:
 [[  3  55]
 [ 18 193]]


## 6. Conclusiones

- El modelo logra predecir el sexo de la víctima con una precisión aceptable basada en variables como edad, tipo de vía, clima y tipo de vehículo.
- Variables como "Edad" y "Vehículo" parecen tener mayor peso en la predicción.
- Este enfoque puede ampliarse para detectar sesgos de género, zonas de riesgo o situaciones de mayor vulnerabilidad.
- El modelo se puede mejorar usando validación cruzada o algoritmos más avanzados como Random Forest.